In [7]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

from data_manipulation_func import load_epex_data

df = load_epex_data('..\data\spot_prices_2025_raw.csv')
df.head()

,MTU (CET/CEST),Area,Sequence,Day-ahead Price (EUR/MWh),Intraday Period (CET/CEST),Intraday Price (EUR/MWh)
0,01/01/2025 00:00:00 - 01/01/2025 01:00:00,BZN|SI,Without sequence,118.46,NaN,NaN
1,01/01/2025 01:00:00 - 01/01/2025 02:00:00,BZN|SI,Without sequence,129.07,NaN,NaN
2,01/01/2025 02:00:00 - 01/01/2025 03:00:00,BZN|SI,Without sequence,121.10,NaN,NaN
3,01/01/2025 03:00:00 - 01/01/2025 04:00:00,BZN|SI,Without sequence,94.28,NaN,NaN
4,01/01/2025 04:00:00 - 01/01/2025 05:00:00,BZN|SI,Without sequence,63.69,NaN,NaN
